In [3]:
import numpy as np
import pandas as pd
import polars as pl

In [3]:
def to_rank(
    primary_key: pd.Series, score: pd.Series, ascending: bool = True, k: int = 100
) -> pd.Series:
    df = pd.DataFrame({primary_key.name: primary_key, score.name: score}).set_index(
        primary_key.name, drop=True
    )
    df = df.sort_values(by=str(score.name), ascending=ascending)
    df["rank"] = np.ceil(np.arange(1, len(df) + 1) / len(df) * k).astype(int)
    return df["rank"]

In [4]:
to_rank(
    pd.Series([1, 2, 3, 4, 5], name="pk"),
    pd.Series([1, 2, 3, 4, 5], name="score"),
    ascending=True,
    k=3,
)

pk
1    1
2    2
3    2
4    3
5    3
Name: rank, dtype: int64

In [5]:
df = pl.DataFrame(
    {
        "pk": [1, 2, 3, 4, 5],
        "score": [1, 2, 3, 4, 5],
    }
)

In [25]:
def to_rank(
    primary_key: pl.Series, score: pl.Series, descending: bool = False, k: int = 100
) -> pl.Series:
    df = pl.DataFrame({primary_key.name: primary_key, score.name: score})
    df = df.sort(by=str(score.name), descending=descending)
    df = df.with_columns(
        pl.Series(
            name="rank",
            values=np.ceil(np.arange(1, len(df) + 1) / len(df) * k),
            dtype=pl.Int64,
        )
    )
    return df["rank"]

In [34]:
to_rank(
    pl.Series("pk", [1, 2, 3, 4, 5]),
    pl.Series("score", [1, 2, 3, 4, 5]),
    descending=True,
    k=3,
)

rank
i64
1
2
2
3
3


In [ ]:
df_1 = pl.DataFrame(
    {
        "pk": [1, 2, 3, 4, 5],
        "score_1": [1, 2, 3, 4, 5],
    }
)

df_2 = pl.DataFrame(
    {
        "pk": [5, 4, 3, 2, 1],
        "score_2": [1, 2, 3, 4, 5],
    }
)

In [9]:
df_2.join(df_1, on="pk", how="left")

pk,score_2,score_1
i64,i64,i64
5,1,5
4,2,4
3,3,3
2,4,2
1,5,1
